This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_predicted_vs_true(train_quad, test_quad, outdir, label):
    """
    Method to create a parity plot (predicted vs. true values)

    Args:

        train_quad: (tuple), tuple containing 4 numpy arrays: true training y data, predicted training y data,

        training metric data, and groups used in training

        test_quad: (tuple), tuple containing 4 numpy arrays: true test y data, predicted test y data,

        testing metric data, and groups used in testing

        outdir: (str), path to save plots to

        label: (str), label used for axis labeling

    Returns:

        None

    """
    filenames = list()
    y_train_true, y_train_pred, train_metrics, train_groups = train_quad
    y_test_true, y_test_pred, test_metrics, test_groups = test_quad

    # make diagonal line from absolute min to absolute max of any data point
    # using round because Ryan did - but won't that ruin small numbers??? TODO this
    max1 = max(y_train_true.max(), y_train_pred.max(),
               y_test_true.max(), y_test_pred.max())
    min1 = min(y_train_true.min(), y_train_pred.min(),
               y_test_true.min(), y_test_pred.min())
    max1 = round(float(max1), rounder(max1-min1))
    min1 = round(float(min1), rounder(max1-min1))
    for y_true, y_pred, stats, groups, title_addon in \
            (train_quad+('train',), test_quad+('test',)):

        # make fig and ax, use x_align when placing text so things don't overlap
        x_align=0.64
        fig, ax = make_fig_ax(x_align=x_align)

        # set tick labels
        # notice that we use the same max and min for all three. Don't
        # calculate those inside the loop, because all the should be on the same scale and axis
        _set_tick_labels(ax, max1, min1)

        # plot diagonal line
        ax.plot([min1, max1], [min1, max1], 'k--', lw=2, zorder=1)

        # do the actual plotting
        if groups is None:
            ax.scatter(y_true, y_pred, color='blue', edgecolors='black', s=100, zorder=2, alpha=0.7)
        else:
            handles = dict()
            unique_groups = np.unique(np.concatenate((train_groups, test_groups), axis=0))
            log.debug(' '*12 + 'unique groups: ' +str(list(unique_groups)))
            colors = ['blue', 'red', 'green', 'purple', 'orange', 'black']
            markers = ['o', 'v', '^', 's', 'p', 'h', 'D', '*', 'X', '<', '>', 'P']
            colorcount = markercount = 0
            for groupcount, group in enumerate(unique_groups):
                mask = groups == group
                log.debug(' '*12 + f'{group} group_percent = {np.count_nonzero(mask) / len(groups)}')
                handles[group] = ax.scatter(y_true[mask], y_pred[mask], label=group, color=colors[colorcount],
                                            marker=markers[markercount], s=100, alpha=0.7)
                colorcount += 1
                if colorcount % len(colors) == 0:
                    markercount += 1
                    colorcount = 0
            ax.legend(handles.values(), handles.keys(), loc='lower right', fontsize=12)

        # set axis labels
        ax.set_xlabel('True '+label, fontsize=16)
        ax.set_ylabel('Predicted '+label, fontsize=16)

        plot_stats(fig, stats, x_align=x_align, y_align=0.90)

        filename = 'predicted_vs_true_'+ title_addon + '.png'
        filenames.append(filename)
        fig.savefig(join(outdir, filename), dpi=DPI, bbox_inches='tight')

    return filenames


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
train_quad = (array([177.28302038, 150.37102595, 202.0924573 , ...,  47.20790294,
        23.809066  ,   0.        ]), array([ 88.64146792,  75.18551297, 101.04618047, ...,  23.60395147,
        11.904533  ,   0.        ]), OrderedDict([('R2', 0.9978171776039921), ('root_mean_squared_error', 71.68448710387179), ('mean_absolute_error', 52.89722610223462), ('rmse_over_stdev', 0.7410283826677081)]), None)
test_quad = (array([2.01402113e+02, 3.44799275e+02, 2.53584855e+02, 7.37475037e+01,
       1.88139584e+02, 1.40791488e+02, 1.11989369e+02, 1.21012896e+02,
       1.44746721e+02, 1.22487016e+02, 2.00504538e+02, 1.58791749e+02,
       1.87709915e+02, 1.53899276e+02, 1.25081235e+02, 3.81107807e+02,
       1.19298374e+02, 9.33858524e+01, 8.38092566e+01, 2.28914531e+02,
       1.87519625e+02, 1.31350152e+02, 4.50731986e+01, 2.35011420e+01,
       1.63185749e+02, 1.71608093e+02, 1.21368422e+02, 2.93743989e+02,
       8.39207969e+01, 9.90662209e+01, 9.70462162e+01, 1.11442294e+02,
       7.30199956e+01, 2.09869299e+02, 1.32814981e+02, 6.71398750e+01,
       7.86712295e+01, 9.25437335e+01, 1.36316077e+02, 1.75379626e+02,
       1.02036436e+02, 6.33627358e+01, 0.00000000e+00, 6.43729766e+02,
       1.27681884e+02, 7.66704600e+00, 5.73220058e+01, 1.56627930e+02,
       6.36890317e+01, 6.78471000e-01, 5.61529988e+01, 5.66738367e+01,
       7.38023139e+01, 1.97421378e+02, 0.00000000e+00, 6.46627457e+01,
       5.35777138e+01, 7.45861041e+01, 0.00000000e+00, 5.38289790e+01,
       1.05881855e+01, 2.26412639e+01, 1.77633032e+02, 7.47720530e+01,
       1.39599684e+02, 1.27825789e+02, 1.49850095e+02, 2.88486563e+02,
       1.50472708e+02, 2.94893175e+02, 7.51559558e+01, 0.00000000e+00,
       7.12267570e+01, 3.39191772e+01, 4.31928159e+01, 5.90546828e+01,
       2.79926398e+01, 4.70181371e+01, 6.21679659e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 3.60810735e+00, 5.93437233e-01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.99014786e+01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.39370004e+02, 2.53695907e+01, 3.35683107e+01, 2.23715509e+01,
       1.35168013e+02, 2.63184751e+01, 0.00000000e+00, 6.37138607e+00,
       6.50312714e+01, 4.02544398e+01, 4.98477351e+01, 4.02646946e+01,
       0.00000000e+00, 3.13433527e+01, 1.89644510e+02, 1.94268537e+02,
       1.02514386e+02, 1.00457045e+02, 1.28935952e+02, 4.22958931e+01,
       3.02287500e+01, 5.05625634e+00, 3.43690177e+01, 8.99135404e+01,
       4.44184734e+01, 3.41493981e+01, 3.01702447e+01, 7.96465197e+01,
       4.54332140e+01, 4.72692617e+01, 2.91224504e+01, 2.36626028e+01,
       0.00000000e+00, 0.00000000e+00, 1.04542615e+02, 1.19114168e+02,
       1.50999876e+02, 9.39602340e+01, 1.24630840e+02, 8.65356568e+01,
       0.00000000e+00, 3.50982856e+00, 1.00949225e+02, 1.29323672e+02,
       1.75579596e+02, 1.85404211e+02, 7.31812963e+01, 1.26881899e+02,
       0.00000000e+00, 1.50078284e+01, 1.93409720e+01, 1.24480773e+01,
       3.29412986e+01, 2.76028721e+02, 2.32454011e+02, 4.84645306e+01,
       2.51360287e+02, 2.94415703e+02, 7.78684422e+01, 1.65778433e+02,
       1.39763133e+02, 2.99692485e+02, 6.24940495e+01, 0.00000000e+00,
       2.50244873e+01, 2.53031123e+01, 2.23740347e+01, 4.22355880e+01,
       4.08070147e+01, 9.53324888e+01, 3.36549838e+01, 1.00440195e+02,
       1.40097275e+00, 0.00000000e+00, 0.00000000e+00, 5.44675418e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.13197137e+01,
       5.25651706e+01, 5.25266742e+01, 4.49236078e+01, 4.71419734e+01,
       4.81837797e+01, 0.00000000e+00, 0.00000000e+00, 5.40734041e+01,
       9.00043474e+01, 2.02393592e+02, 3.81230146e+01, 2.62753383e+01,
       4.31541225e+01, 8.26646597e+01, 1.57158326e+02, 1.29976759e+02,
       5.07080997e+01, 6.08628852e+01, 4.16645268e+01, 4.12956968e+01,
       4.07604224e+01, 5.20549201e+01, 2.14364169e+02, 0.00000000e+00,
       1.28386129e+02, 1.06701247e+02, 1.20596615e+02, 8.10394982e+01,
       2.44136176e+01, 2.57130076e+01, 8.22953902e+01, 7.16777317e+01,
       5.29102783e+01, 7.01397816e+01, 1.37881939e+02, 8.58908577e+01,
       8.06894184e+01, 1.89735033e+01, 5.71443815e+01, 8.63070905e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.40607889e+01,
       0.00000000e+00, 7.56564178e+00, 4.21875225e+01, 1.68517827e+02,
       1.79412889e+02, 1.82075421e+02, 2.15604850e+02, 1.89093084e+02,
       1.79654132e+02, 2.86026589e+02, 1.65118221e+02, 1.68867302e+02,
       1.57972803e+02, 1.58037965e+02, 1.86400275e+02, 2.13314660e+02,
       2.87661839e+02, 3.67138367e+02, 3.36224697e+02, 1.16180977e+02,
       1.78939785e+02, 1.10139318e+02, 1.14272223e+02, 1.42295818e+02,
       1.96074159e+02, 1.03756755e+02, 1.00207954e+02, 1.69625113e+02,
       1.40405957e+02, 1.06220359e+02, 2.31128112e+02, 1.18248560e+02,
       3.57145351e+02, 2.22246511e+02, 2.26507616e+02, 2.06364164e+02,
       2.23816699e+02, 3.20529825e+02, 3.40437655e+02, 2.13493682e+02,
       1.01887892e+02, 1.56185442e+02, 7.08220872e+01, 1.35164757e+02,
       1.66212813e+02, 2.47684975e+02, 1.51356144e+02, 2.96095913e+02,
       8.84773614e+01, 1.35408933e+02, 1.92010418e+02, 3.29042834e+02,
       2.38522270e+02, 3.30939913e+02, 2.50995845e+02, 2.58640047e+02,
       3.50052571e+02, 3.49605405e+02, 2.47474247e+02, 8.95779086e+01,
       1.04459645e+02, 1.24842572e+02, 8.85969515e+01, 1.50042028e+02,
       9.73721500e+01, 2.23624761e+02, 1.39928045e+02, 1.47525298e+02,
       1.62784213e+02, 1.38091226e+02, 5.46384029e+01, 1.43519281e+02,
       7.28852659e+01, 1.84330790e+02, 8.68301581e+01, 1.17792277e+02,
       1.25571144e+02, 1.78519868e+02, 2.27717761e+02, 2.47909595e+02,
       3.04851246e+02, 2.95582043e+02, 2.87755496e+02, 2.51703483e+02,
       3.18785354e+02, 1.61637650e+02, 1.81019902e+02, 8.42083837e+01,
       1.01883876e+02, 8.21190201e+01, 1.41354043e+02, 8.46282081e+01,
       9.54271066e+01, 1.25309562e+02, 1.22028582e+02, 8.32175656e+01,
       8.35342519e+01, 1.17389412e+02, 9.94733566e+00, 2.91372971e+01,
       7.09524931e+01, 3.31838908e+01, 7.59835817e+01, 4.15835904e+01,
       3.27734114e+01, 5.83780479e+01, 2.05075217e+02, 1.62590223e+02,
       1.49540965e+02, 1.70292754e+02, 1.65127413e+02, 1.92128669e+02,
       2.18074203e+02, 1.85543114e+02, 1.22370956e+02, 1.71603969e+02,
       1.24542111e+02, 1.08537384e+02, 1.06745759e+02, 1.13060644e+02,
       1.13886204e+02, 1.02775420e+02, 2.58828051e+02, 1.22316043e+02,
       1.17249168e+02, 2.96630230e+02, 1.84901449e+02, 1.84591316e+02,
       1.85661694e+02, 1.64660391e+02, 2.91099073e+02, 1.12287386e+02,
       1.13501499e+02, 1.10864718e+02, 2.98245707e+02, 1.41965660e+02,
       9.12669747e+01, 9.07487562e+01, 8.28791006e+01, 1.07783426e+02,
       1.82613120e+02, 4.21335070e+01, 6.69631063e+01, 1.83475519e+02,
       2.90221571e+02, 2.71330052e+01, 1.18947119e+02, 1.51299348e+01,
       3.13127509e+01, 1.12949106e+02, 7.84670662e+01, 4.06890701e+01,
       6.35864630e+01, 1.07860026e+02, 1.82417160e+02, 3.87496682e+01,
       9.72556960e+01, 4.95323894e+01, 0.00000000e+00, 0.00000000e+00,
       6.41409237e+01, 1.78464077e+01, 1.13483054e+02, 4.92192870e+01,
       5.92192098e+01, 5.41514922e+01, 3.42214622e+01, 0.00000000e+00,
       0.00000000e+00, 2.84070013e+01, 2.71196539e+01, 3.17633706e+01,
       2.99157066e+01, 3.77846853e+01, 7.07659236e+01, 2.01387280e+01,
       7.26501274e+01, 7.51456208e+01, 7.90798034e+01, 1.21693503e+02,
       9.11059876e+01, 3.81925114e+01, 5.25504455e+01, 3.91274706e+01,
       4.02393064e+01, 0.00000000e+00, 1.03731415e+02, 1.52570887e+02,
       1.23849520e+02, 1.66347669e+02, 0.00000000e+00, 7.33023683e+01,
       1.10664774e+02, 0.00000000e+00, 8.78388717e+01, 5.78409146e+01,
       2.71567126e+01, 2.07374692e+01, 5.90626223e+01, 1.60395108e+02,
       5.40260430e+01, 0.00000000e+00, 3.53717988e+01, 4.50623816e+01,
       2.28817925e+01, 9.28082416e+01, 9.90236861e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.58851259e+00, 0.00000000e+00,
       0.00000000e+00]), array([0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 3.15225147e+001, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 5.93645751e-199, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       3.81945241e-199, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 1.74782432e-316, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       2.43713856e+001, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000]), OrderedDict([('R2', 0.0005964407348102441), ('root_mean_squared_error', 139.39573123371736), ('mean_absolute_error', 106.5112606960092), ('rmse_over_stdev', 1.4409839205130466)]), None)
outdir = '/home/nerve/Desktop/skunkworks/lane_schultz/hyper_parameters/test/DoNothing/DoNothing/KernelRidge_DoNothing_DoNothing_GridSearch/RepeatedKFold/split_1'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_paths = plot_predicted_vs_true(train_triple, test_triple, outdir)
for plot_path in plot_paths:
    display(Image(filename=plot_path))
